# Using a Neural Network on the Breast Cancer Wisconsin Dataset

Let's note that I'm a student in one of Microsoft AI School in France and that I am experimenting one of my first Neural Network on Keras.

I did try first to use the MLP Classifier from sklearn [in this Notebook](https://www.kaggle.com/mizujou/breast-cancer-mlpclassifier-by-mizujou), and I want to compare the results with a simple NN with Keras.

---

# 1. Import Librairies

In [ ]:
# Basics
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import IPython

# Tensorflow
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
from tensorflow.keras import callbacks
from tensorflow.keras.layers.experimental import preprocessing
import kerastuner as kt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

# Graphs
import matplotlib.pyplot as plt
import seaborn as sns

print("Numpy: " + np.__version__)
print("Tensorflow: " + tf.__version__)
print("Keras: " + keras.__version__)

---

# 2. Loading the Dataset

Attributes information:
* Diagnosis (0 = malignant, 1 = benign)

In [ ]:
X, y = load_breast_cancer(return_X_y=True)
X.shape, y.shape

---

# 3. Split the Dataset

In [ ]:
train_data, test_data, train_targets, test_targets = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=0, 
    shuffle=True,
    stratify=y
)

train_data.shape, train_targets.shape, test_data.shape, test_targets.shape

## Simple EDA

In [ ]:
train_df = pd.DataFrame(train_data)
train_df.head(3)

'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
    'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean',
    'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se',
    'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se',
    'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se',
    'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
    'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst',
    'symmetry_worst', 'fractal_dimension_worst'

In [ ]:
sns.pairplot(train_df[:], diag_kind='kde')

---

# 4. HyperTuning

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.BatchNormalization(input_shape=[30]))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=2, max_value=30, step=2)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) 

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=keras.losses.BinaryCrossentropy(), 
      metrics=['binary_accuracy']
  )

  return model

In [ ]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_binary_accuracy', 
    max_epochs=10,
    factor=3,
)   

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(
    train_data, train_targets, 
    epochs=1500, 
    validation_split=0.1, # We set the size of our validation set here
    callbacks=[ClearTrainingOutput()]
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

---

# 5. Creation of the model

1. We will start by trying to recreate the architecture of the best model using MLPClassifier and see the performance

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
    restore_best_weights=True,
)

model = keras.Sequential([
    layers.BatchNormalization(input_shape=(30,)),
    layers.Dense(14, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001), 
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

model.summary()

In [ ]:
history = model.fit(
    train_data, train_targets,
    validation_split=0.1,
    batch_size=None,
    epochs=500,
    callbacks=[early_stopping],
    verbose=1, # hide the output because we have so many epochs
)

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[5:, ['loss', 'val_loss']].plot()
history_df.loc[5:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_binary_accuracy'].max()))


---

# 6. Performance on the Test Dataset

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_targets, verbose=2)

print('\nTest accuracy:', test_acc)

---

# 8. Conclusion

* We successfully find the same Accuracy with Keras and our previous [MLP Classifier version](https://www.kaggle.com/mizujou/breast-cancer-mlpclassifier-by-mizujou) which is **95,61%**
* Once again, the point of this notebook wasn't to find the best model using Keras but more to have a first approach using Keras with a simple architecture of a Neural Network

### Update

* After launching the notebook again, I decided to change some things like the size of the validation split since it is a small dataset which even made the model generalize better and finish now with a **98%** accuracy on the Test Set.

